In [1]:
import numpy as np
import random as random
import matplotlib.pyplot as plt
import time
import pandas as pd
import math
from scipy.spatial import distance
from scipy.stats import mode
from pyproj import Proj,transform
import seaborn as sns
import copy

# Average Farm Sizes (no. sheep and no. cow)

In [4]:
Data=pd.read_csv("all_farms/All_data",sep = '\t', header = 0)

FileNotFoundError: File b'all_farms/All_data' does not exist

In [2]:
psi = 0.00657
# psi = 2.5
nu = 1.99*(10**(-4.8))
xi = 4.65
zeta = 2.80
chi = 0.403
phi = 0.799
rho = 0.000863
# epsilon = np.zeros(N)
# s = np.random.negative_binomial(50, 50/55, N) #Draw latent periods (5 days)
# r = np.random.negative_binomial(30, 30/38, N) #Draw infectious periods (8 days)
# q = np.random.negative_binomial(2*28, 2*28/(4*28), N) #Draw immune periods (2 months)
# v = np.random.negative_binomial(6*28, 6*28/(12*28))

In [ ]:
cattle = copy.deepcopy(df['cattle'].values).astype(int)
sheep = copy.deepcopy(df['sr'].values).astype(int)
t = 0
cost = np.zeros(n)
A = np.zeros(shape = (n, 10))
A[:,0] = -1 
I = np.zeros(n)
vac = np.zeros(n)

cumInf = np.zeros(tmax+1)
numInf = np.zeros(tmax+1)
endemic = 0


indent = 1
for k in range(num_initial):
    if k ==0:
        initial = random.randint(0,n-1)
        I[initial] = 1
        A[initial, ] = [initial, 0, s[initial], r[initial], tr[initial], q[initial], 0,0,  0,0]
    else:
        initial2 = (initial + indent)%N
        I[initial2] = 1
        A[initial2, ] = [initial2, 0, s[initial2], r[initial2], tr[initial2], q[initial2], 0, 0, 0, 0]
        indent +=1 

infect_cow = np.zeros(n)
infect_sheep = np.zeros(n)


susceptind = [i for i in range(np.size(I)) if I[i] ==0]
exposedind = [i for i in range(np.size(I)) if I[i] == 1]
infectind = [i for i in range(np.size(I)) if I[i]==2]
vaccineind = [i for i in range(np.size(I)) if I[i]==3]
immuneind = [i for i in range(np.size(I)) if I[i] ==4]

time_plot=[0]
Inf=[len(infectind)]
Sus = [len(susceptind)]
Exp = [len(exposedind)]
Imm = [len(immuneind)]
Vac = [len(vaccineind)]

Dose = []
VAC = []
travelled = []

while t < tmax:
    t = t+1
    print(t)
    infNum = 0

    ################################### WITHIN FARM INFECTION ####################################
    for farm in infectind:
            Num_livestock = infect_cow[farm] + infect_sheep[farm]
            inf = 1-((1-λ)**Num_livestock) 
            for animals in range(int(cattle[farm]+sheep[farm])):
                p_beta = np.random.uniform(0,1)
                if (p_beta<inf) and (infect_sheep[farm]+infect_cow[farm])<=Num_livestock:
                    if (random.random() < prob_sheep and sheep[farm]>=1 and infect_sheep[farm]<sheep[farm]):
                        infect_sheep[farm] += 1
                    elif (cattle[farm]>=1 and infect_cow[farm]<cattle[farm]):
                        infect_cow[farm] +=1

    ################################# BETWEEN FARM INFECTION  #####################################

    ###############################################################################################
    ######################################### ATTRIBUTES ##########################################
    ###############################################################################################
    Attribute_matrix = trans_M.copy()
    beta = np.zeros(n)
    beta1 = nu*(xi*(cattle-infect_cow)**chi + (sheep-infect_sheep)**chi)
    beta3 = zeta*(infect_cow)**chi + infect_sheep**chi
    beta2 = sparse.diags(beta3, shape = (n, n))


    Attribute_matrix = sparse.csr_matrix.dot(Attribute_matrix, beta2)

    for j in range(0,n):
        a = Attribute_matrix[j, I ==2]
        dist = np.array([math.sqrt(sum((joinedinput[j] - i)**2))/(100000) for i in joinedinput[I==2]])

        b = kerneldist(dist)

        beta[j] = beta1[j]*sparse.csr_matrix.dot(a,b)

    prob_inf = (1 - np.exp(-beta))
    unif = np.random.uniform(0, 1, n)

    ### immunity ###
    for i in range(n):
        if (I[i] == 4) or (I[i] == 3):
            cattle[i] = 0
            sheep[i] = 0
            prob_inf[i] = 0
    for i in range(0,n):
        if (unif[i] <= prob_inf[i] and I[i] == 0):
            cattle[i] = true_cattle[i]
            sheep[i] = true_sheep[i]
            I[i] =  1
            A[i, ] = [i, t, s[i], r[i], tr[i], q[i], 0, 0,0,0]

    ###################################### EXPOSED TO INFECTIOUS ############################################

    inf = A[:,0][A[:,1] + A[:,2] == t]

    I[inf.astype(np.int64)] = 2


    ####################### UPDATE NUMBER OF INFECTED ANIMALS WITHIN AN INFECTIOUS FARM #####################

    for i in inf: 
        i = int(i)
        if (random.random() < (sheep[i]/(sheep[i] +cattle[i])) and sheep[i]>=1 and infect_sheep[i]<sheep[i]):
            infect_sheep[i] += 1
        elif (cattle[i]>=1 and infect_cow[i]<cattle[i]):
            infect_cow[i] +=1

    ########################################### RING VACCINATION ###########################################

    trig = A[:,0][A[:,1] + A[:,2] + A[:,4] == t] #trigger vaccines
    RingVac = ring_vac/100
    for i in trig:
        i = int(i)
        dist = np.array([math.sqrt(sum((joinedinput[i] - joinedinput[j])**2))/(100000) for j in range(n)])
        n_vac = [k for k in range(n) if dist[k] <RingVac]
        index=list(np.where(np.random.uniform(0,1,len(n_vac))<ring_cover*np.ones(len(n_vac)))[0])
        ind_vac=list(np.asarray(n_vac)[index])

        for j in range(len(ind_vac)):
            m = ind_vac[j]
            if A[m,7] == 0:
                VAC = np.append(VAC, m)
                travelled.append(dist[m])

    VAC, indices = np.unique(np.array(VAC, dtype=np.int), return_inverse=True)
    VAC = VAC[indices]
    if len(VAC) > 0:
        VAC = VAC.astype(np.int64)
    if len(VAC)>Capacity:
        cost[VAC[0:Capacity]] += farm_cost[VAC[0:Capacity]]

        for j in range(len(VAC[0:Capacity])):
            m = VAC[j]
            A[m,0] = m
            A[m,7] = t
            if (I[m] == 0):

                if np.random.uniform(0,1) <efficacy:
                    A[m, 8] = np.random.negative_binomial(6*28, 6*28/(12*28))
                    A[m,9] = A[m,8] + t
                    I[m] = 3
                    cattle[m] = 0
                    sheep[m] = 0
            elif (I[m] == 3):
                if np.random.uniform(0,1) < efficacy:
                    immunity = np.random.negative_binomial(6*28, 6*28/(12*28))
                    if (A[m,9]) < (immunity+t):
                        A[m,8] = immunity
                        A[m, 9] = immunity +t
                    else:
                        A[m,8] = A[m,9] - t
            elif (I[m] ==4):
                if np.random.uniform(0,1) < efficacy:
                    if (A[m,6] )< (v +t):
                        A[m,8] = v
                        A[m,9] = v + t
                        I[m] = 3
                        A[m,6] = 0
                        cattle[m] = 0
                        sheep[m] = 0
        VAC = np.delete(VAC, range(0, Capacity), None)
    else: 
        cost[0:len(VAC)] += farm_cost[0:len(VAC)]

        for j in range(len(VAC)):
            m = VAC[j]
            A[m, 0] = m
            A[m, 7] = t
            if (I[m] == 0):
                if np.random.uniform(0,1) <efficacy:
                    A[m, 8] = np.random.negative_binomial(6*28, 6*28/(12*28))
                    A[m, 9] = A[m, 8] +t
                    I[m] = 3
                    cattle[m] = 0
                    sheep[m] = 0
            elif (I[m] == 3):
                if np.random.uniform(0,1) < efficacy:
                    immunity = np.random.negative_binomial(6*28, 6*28/(12*28))
                    if (A[m,9]) < (immunity +t):
                        A[m,8] = immunity
                        A[m,9] = immunity +t
                    else:
                        A[m,8] = A[m,9] - t
            elif (I[m] ==4):
                if np.random.uniform(0,1) < efficacy:
                    immunity = np.random.negative_binomial(6*28, 6*28/(12*28))
                    if (A[m,6] )< (immunity +t):
                        A[m,8] = immunity
                        A[m,9] = immunity + t
                        I[m] = 3
                        A[m,6] = 0  
                        cattle[m] = 0
                        sheep[m] = 0
        VAC = np.delete(VAC, range(0, len(VAC)), None)

    ########################################### vaccine wears off ###########################################

    new_dose = A[:,0][A[:,7]+6*28 == t] #records when the farmer will get a new vaccine


    for j in new_dose:
        j = j.astype(np.int64)
        if (A[j,8] > 6*28) and (I[j]==3):
            A[j, 7] = 0
        else: 
            A[j, 7] = 0
            A[j, 8] = 0
            A[j,9] = 0
            if I[j] == 3:
                I[j] = 0
                cattle[j] = true_cattle[j]
                sheep[j] = true_sheep[j]

    vaccinate_ends = A[:,0][A[:,9] == t]

    for j in vaccinate_ends:
        j = j.astype(np.int64)
        if I[j] ==3:
            I[j] =0
            cattle[j] = true_cattle[j]
            sheep[j] = true_sheep[j]

    ######################################### IMMUNITY #######################################################

    rem = A[:,0][A[:,1] + A[:,2] + A[:,3] == t] #Move to IM state once infectious period is over
    for i in rem:
        i = i.astype(np.int64)
        if I[i] == 2:
            I[i] =4
            cattle[i] = 0
            sheep[i] = 0
            A[i, 6] = A[i, 5] +t
            infect_sheep[i] = 0
            infect_cow[i] = 0

    immune = A[:,0][A[:,1] + A[:,2] + A[:,3] + A[:,5] == t] #Move to S state once immune period is over
    I[immune.astype(np.int64)] = 0
    A[immune.astype(np.int64), ] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for k in immune:
        k = k.astype(np.int64)
        cattle[k] = true_cattle[k]
        sheep[k] = true_sheep[k]


    #### WITHIN FARM RECOVERY ####
    infected = [i for i in range(n) if (infect_cow+infect_sheep)[i]>0]
    for farm in infected:
        recover = np.random.uniform(0,1)
        for animals in range(int(infect_cow[farm]+infect_sheep[farm])):
            if recover < μ and (infect_sheep[farm]+infect_cow[farm]) >=1 :
                if (random.random() <(infect_sheep[farm]/(infect_sheep[farm]+infect_cow[farm])) and infect_sheep[farm]>=1 ):
                    infect_sheep[farm] -= 1
                elif infect_cow[farm]>=1: 
                    infect_cow[farm] -=1
                if infect_sheep[farm]+infect_cow[farm] ==0: 
                    I[farm] = 4
                    infect_sheep[farm] = 0
                    infect_cow[farm] = 0
                    cattle[farm] = 0
                    sheep[farm] = 0
                    A[farm, 6] = A[farm, 5] +t
    ################################### end of each day #########################################
    infectind = [i for i in range(np.size(I)) if I[i]==2]
    susceptind = [i for i in range(np.size(I)) if I[i]==0]
    exposedind = [i for i in range(np.size(I)) if I[i] == 1]
    immuneind = [i for i in range(np.size(I)) if I[i] ==4]
    vaccineind = [i for i in range(np.size(I)) if I[i]==3]
    had_dose = [i for i in range(np.size(I)) if A[i,6]>0]
    if t>0:
        infNum += len(inf)

    numinf2 = len(inf)
    Dose.append(len(had_dose))
    Inf.append(len(infectind))
    Sus.append(len(susceptind))
    Exp.append(len(exposedind))
    Imm.append(len(immuneind))
    Vac.append(len(vaccineind))
    time_plot.append(t)
    cumInf[t] = cumInf[t-1] + infNum
    numInf[t] = numinf2

    if sum(I == 1) + sum(I == 2) == 0:
        a = cumInf[t-1]
        cumInf[t:] = a
        numInf[t:] = 0 
        endemic = 0
        total_numinf = np.max(Inf)
        break
    if t == (tmax-1):
        endemic = np.mean(Inf[(tmax-100):])
        total_numinf = np.max(Inf)
        break
